# Sentiment Prediction Server

In [ ]:
# !pip install flask_cors
# !pip install flask

In [ ]:
import joblib
from flask import Flask
from flask import jsonify, request
from flask_cors import CORS

In [ ]:
def load_model_file(filename):
    pipeline = joblib.load(filename)
    return pipeline

def predict(model, sentences):
    sentences = sentences if isinstance(sentences, list) else [sentences]
    pred = model.predict(sentences)

    return ['positive' if x == 1 else 'negative' for x in pred]

In [ ]:
model = load_model_file('movie_model.pkl')
model

In [ ]:
predict(model, 'this is a great movie.')

In [ ]:
predict(model, 'this is a horrible movie.')

In [ ]:
predict(model,[
    'funny movie',
    'awesome movie',
    'stupid movie'
])

In [ ]:
import urllib

ip = '127.0.0.1:5000'
sentence_encoded = urllib.parse.quote('this is a good movie')
test_url = f'http://{ip}/?sentence={sentence_encoded}'

In [ ]:
app = Flask(__name__)
CORS(app)

@app.route('/')
def start():
    sentence = request.args.get('sentence')

    if sentence == None: 
        return jsonify({})

    print(sentence)
    
    result = predict(model, sentence)
    
    # print(result)
    return jsonify({
        'sentence': sentence,
        'sentiment': result[0]
    })


if __name__ == '__main__':
    print('test url: ', test_url)
    app.run()


In [ ]:
# for mac/linux to kill running server on port 5000
!lsof -ti:5000 | xargs kill -9

# For production

In [ ]:
!cat server.py

Let's copy the server script and model file to S3

In [ ]:
!aws s3 cp movie_model.pkl s3://com.msarica.ds/movie_model.pkl
!aws s3 cp server.py s3://com.msarica.ds/server.py